In [ ]:
# 보고서 생성 : 이미지 추가
# 출처 : https://www.slideshare.net/plusjune/ss-110416117
# 슬라이드 내 37page

###  설정 내용
+ [x] 이메일 발송 모듈 만들기 
+ [x] 이메일 양식(내용) frame 구성하기
+ [x] 발송 셋팅
  + [x] 이메일 양식 관련 변수 설정
    + [x] 발송일자 설정
    + [x] 표 붙여넣기
      + [ ] 동적 효과 적용
      + [ ] 변동 내용 추가
    + [ ] 이미지 붙여넣기
  + [x] 발송 변수 설정
    + [x] mulit 발송
    + [x] 첨부파일 추가
+ [x] 이메일 발송하기(모듈 실행)

In [ ]:
## 이메일 발송 모듈 만들기

import pandas as pd
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders
from email.header import Header
from datetime import datetime, timedelta

import sqlite3

from jinja2 import Template


gmail_user='************'
gmail_pwd ='************'

def send_gmail(to, subject, html, attach=None):
    msg=MIMEMultipart('alternative')
    msg['From']=gmail_user
    msg['To'] = to
    msg['Subject'] = Header(s=subject, charset="utf-8")
    msg.attach(MIMEText(html, 'html', _charset="utf-8"))

    #첨부파일
    if attach:
        part=MIMEBase('application','octet-stream')
        part.set_payload(open(attach, 'rb').read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition','attachment; filename="%s"' % os.path.basename(attach))
        msg.attach(part)
     
    s = smtplib.SMTP("smtp.gmail.com", 587)
    s.ehlo()
    s.starttls()
    s.ehlo()
    s.login(gmail_user, gmail_pwd)
    s.sendmail(gmail_user, to, msg.as_string())
    s.close()

In [ ]:
# 이메일 양식(내용) frame 구성하기
message_tmpl="""<html>
<!DOCTYPE html>
<html>
<head lang ="ko">
  <meta charset ="UTF-8">
  <title>{{title1}}</title>
</head>
<body>
안녕하세요, CMR팀 박경하 입니다.<br/>
<br/>
지난주({{d0}} ~{{d1}})온라인상에서의 분유가격 동향결과 공유드립니다.<br/>
  
  <h3> {{title1}} </h3>
  {{headline_text}}
  
  <h3> {{title2}} </h3>
  {{title2_text1}}<br/>
  {{ df1_html }}<br/>
  {{title2_text2}}<br/>
  {{ df2_html }}<br/>

  <h3> {{title3}} </h3>
  <br/>
  
<br/><br/>
감사합니다.
</body>
</html>
"""

In [ ]:
# 표 만들기
##.db 파일(스키마) 접속
con = sqlite3.connect("C:/Users/khpark/Google 드라이브/python/분유가격 분석/MONITERING_PRICE_test.db")
cursor = con.cursor()


In [ ]:
STAGE, COMPANY = "1단계", "매일"

s_dt = (datetime.today()- timedelta(8)).strftime('%Y-%m-%d')
e_dt = (datetime.today()- timedelta(1)).strftime('%Y-%m-%d')
# s_dt = '2018-08-10'
# e_dt = '2018-08-20'

# query = ("SELECT A.DATE, B.LINE, min(A.PRICE) as PRICE FROM MILKPOWER_MIN A, MP_PRODUCT B where B.PRODUCT = A.PRODUCT " +
#          'and B.STAGE = '   + "'" + STAGE   + "' " + 
#          'and B.COMPANY = ' + "'" + COMPANY + "' " + 
#          'and A.DATE between '+ "'" + s_dt + "' and '" +e_dt+"' " +
#          "and B.LINE not in ('저가') " + 
#          'group by A.DATE, B.LINE '
#         )

query = ("SELECT distinct A.DATE , C.고가, D.중가, E.민감성, F.산양, G.액상, H.스틱 " +
         "FROM MILKPOWER_MIN A " + 
         "left join (select A.DATE, min(A.PRICE) as 고가   from MILKPOWER_MIN A, MP_PRODUCT B where B.PRODUCT = A.PRODUCT and B.STAGE = '1단계' and B.LINE ='고가'   and B.COMPANY = '매일' group by A.DATE ) C on A.date = C.date "+
         "left join (select A.DATE, min(A.PRICE) as 중가   from MILKPOWER_MIN A, MP_PRODUCT B where B.PRODUCT = A.PRODUCT and B.STAGE = '1단계' and B.LINE ='중가'   and B.COMPANY = '매일' group by A.DATE ) D on A.date = D.date "+
         "left join (select A.DATE, min(A.PRICE) as 민감성 from MILKPOWER_MIN A, MP_PRODUCT B where B.PRODUCT = A.PRODUCT and B.STAGE = '1단계' and B.LINE ='민감성' and B.COMPANY = '매일' group by A.DATE ) E on A.date = E.date "+
         "left join (select A.DATE, min(A.PRICE) as 산양   from MILKPOWER_MIN A, MP_PRODUCT B where B.PRODUCT = A.PRODUCT and B.STAGE = '1단계' and B.LINE ='산양'   and B.COMPANY = '매일' group by A.DATE ) F on A.date = F.date "+
         "left join (select A.DATE, min(A.PRICE) as 액상   from MILKPOWER_MIN A, MP_PRODUCT B where B.PRODUCT = A.PRODUCT and B.STAGE = '1단계' and B.LINE ='액상'   and B.COMPANY = '매일' group by A.DATE ) G on A.date = G.date "+
         "left join (select A.DATE, min(A.PRICE) as 스틱   from MILKPOWER_MIN A, MP_PRODUCT B where B.PRODUCT = A.PRODUCT and B.STAGE = '1단계' and B.LINE ='스틱'   and B.COMPANY = '매일' group by A.DATE ) H on A.date = H.date "+
         'where A.DATE between '+ "'" + s_dt + "' and '" +e_dt+"' "
        )

# query = ("select A.DATE, min(A.PRICE) as 고가 " +
#          "from MILKPOWER_MIN A, MP_PRODUCT B " +
#          "where B.PRODUCT = A.PRODUCT " +
#          "and B.STAGE = '1단계' and B.LINE ='고가' group by A.DATE ")

# 변수지정
# STAGE,PACKAGE,VOLUNM,LINE = "1단계","캔","대","고가"

# 쿼리설정
# query = ("SELECT A.DATE || ' ' || A.TIME as DATE, B.BRAND, min(A.PRICE) as PRICE FROM MILKPOWER_MIN A, MP_PRODUCT B where B.PRODUCT = A.PRODUCT " +
#          'and B.STAGE = '   + "'" + STAGE   + "' " + 
#          'and B.TYPE2 = '   + "'" + PACKAGE + "' " +
#          'and B.VOLUNM2 = ' + "'" + VOLUNM  + "' " +
#          'and B.LINE = '    + "'" + LINE    + "' " +
#          "group by A.DATE, A.TIME, B.BRAND")

# 쿼리실행
df = pd.read_sql(query, con)

#자료 피벗팅
# df = df11.pivot('DATE','LINE','PRICE')
# df11 = pd.pivot_table(df11, values = 'PRICE', index = 'DATE', columns= 'LINE')


In [ ]:
df

,DATE,고가,중가,민감성,산양,액상,스틱
0,2018-08-14,"14,790","10,500","19,190",None,"7,330","11,670"
1,2018-08-15,"14,790","10,500","19,200",None,"7,330","11,670"
2,2018-08-16,"14,790","10,500","19,200","41,300","7,330","11,670"
3,2018-08-17,"14,790","10,500","19,200","41,300",None,"11,670"
4,2018-08-18,"14,790","10,500","19,200","41,300","10,300","11,670"
5,2018-08-19,"14,790","10,930","19,200","41,300","10,300","11,670"
6,2018-08-20,"14,790","10,930","19,200","41,300","27,570","11,670"


In [ ]:
import matplotlib.pyplot as plt
plt.figure(); df.plot();

TypeError: Empty 'DataFrame': no numeric data to plot

In [ ]:
# Pandas : Table Styles

from IPython.display import HTML

def hover(hover_color="#ffff99"):
    return dict(selector="tr:hover",
                props=[("background-color", "%s" % hover_color)])

styles = [
    hover(),
    dict(selector="th", props=[("font-size", "100%"),
                               ("text-align", "center")]),
    dict(selector="caption", props=[("caption-side", "bottom")])
]
df = (df.style.set_table_styles(styles))


In [ ]:
df

LINE,고가,민감성,산양,스틱,액상,중가
DATE,,,,,,
2018-08-14,"14,790","19,190",None,"11,670","7,330","10,500"
2018-08-15,"14,790","19,200",None,"11,670","7,330","10,500"
2018-08-16,"14,790","19,200","41,300","11,670","7,330","10,500"
2018-08-17,"14,790","19,200","41,300","11,670",None,"10,500"
2018-08-18,"14,790","19,200","41,300","11,670","10,300","10,500"
2018-08-19,"14,790","19,200","41,300","11,670","10,300","10,930"
2018-08-20,"14,790","19,200","41,300","11,670","27,570","10,930"


In [ ]:
ax = df.plot()
fig = ax.get_figure()
fig_data = get_fig_base64(fig)

AttributeError: 'Styler' object has no attribute 'plot'

In [ ]:
# 발송일자 설정
from datetime import datetime, timedelta
import sqlite3

now = datetime.today()    # 현재
# now = datetime.today()+ timedelta(9)    # 현재 in google
date = now.strftime('%Y%m%d')

# 그래프 -> 이미지 변환
# df_2017_12 = dpr.DataReader()
# ax = df_2017_12.plot()
# fig = ax.get_figure()
# fig_data = get_fig_base64(fig)

title1 = "1. 지난주 자사 분유 가격 변동(3단계 기준)"
headline_text = "다수의 전문가들은"
title2 = "2. 주로 라인별 분유 가격 추이(3단계 기준)"
title3 = "3. 최근 1개월 분유가격 : 첨부 파일 참조"

render_data = {
    'd0' : (datetime.today()- timedelta(1)).strftime('%m/%d'),
    'd1' : (datetime.today()- timedelta(7)).strftime('%m/%d'),
    'title1' : title1,
    'title2' : title2,
    'headline_text' : headline_text,
    'title2_text1' : " - 궁 일별 단계별 최저가",
#     'df1_html' : fig,
    'title2_text2' : " - 아이엠마더 일별 단계별 최저가",
    'df2_html' : df.to_html(),
    'title3' : title3
#     'imgae_data' = './image/분유_스틱.png'
}

t= Template(message_tmpl)
html_report = t.render(render_data)


In [ ]:
#발송 변수 확인
# date
# df

In [ ]:
# 발송 변수 설정

# 받는사람 : test 발송
to_users = ['박경하(CRM팀) <khpark@maeil.com>']

# 받는사람 : mulit 발송
# to_users = ['박경하(CRM팀) <khpark@maeil.com>', '박경하(CRM팀) <khpark@maeil.com>']

# 메일제목
subject = date + "_분유가격 리포트(발송전용)"

#20180801_분유가격 리포트(발송전용)

# 본문
html = html_report

#첨부파일 추가
attach = None

# attach = './image/.png'

In [ ]:
# 이메일 발송
for to_users in to_users :
    send_gmail(to_users, subject, html, attach)